<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Import-modules" data-toc-modified-id="Import-modules-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Import modules</a></span></li><li><span><a href="#WARP" data-toc-modified-id="WARP-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>WARP</a></span></li><li><span><a href="#Read-in-the-dataset" data-toc-modified-id="Read-in-the-dataset-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Read-in the dataset</a></span></li><li><span><a href="#BPR-vs.-WARP" data-toc-modified-id="BPR-vs.-WARP-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>BPR vs. WARP</a></span></li><li><span><a href="#Conclusions" data-toc-modified-id="Conclusions-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Conclusions</a></span></li><li><span><a href="#References" data-toc-modified-id="References-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>References</a></span></li></ul></div>

# Introduction
<hr style = "border:2px solid black" ></hr>

<div class="alert alert-warning">
<font color=black>

**What?** WARP = Weighted Approximate Rank Pairwise Loss

</font>
</div>

# Import modules
<hr style = "border:2px solid black" ></hr>

In [ ]:
import numpy as np
from time import time
from scipy.stats import geom
from lightfm import LightFM
from lightfm.evaluation import auc_score
from lightfm.evaluation import precision_at_k
from lightfm.datasets import fetch_movielens
# Getting rid of the warning messages
import warnings
warnings.filterwarnings("ignore")

# WARP
<hr style = "border:2px solid black" ></hr>

<div class="alert alert-info">
<font color=black>
    
- Like the Bayesian Personalized Ranking (BPR) model, **WARP** deals with (user, positive item, negative item) triplets. Unlike BPR, the negative items in the triplet are not chosen by random sampling: they are chosen from among those negative items which would violate the desired item ranking given the state of the model. This approximates a form of active learning where the model selects those triplets that it cannot currently rank correctly.

- This procedure yields roughly the following algorithm:

    - For a given user, positive item pair, sample a negative item at random from all the remaining items. Compute predictions for both items; if the negative item's prediction exceeds that of the positive item plus a margin, perform a gradient update so we can rank the positive item higher and the negative item lower. If there is no rank violation, continue sampling negative items until a violation is found.
    - If we found a violating negative example at the first try, make a large gradient update: this indicates that a lot of negative items are ranked higher than positives items in current state of the model, and the model should be updated by a large amount. If it took a lot of sampling to find a violating example, perform a small update since the model is likely close to the optimum and should be updated at a low rate.

</font>
</div>

# Read-in the dataset
<hr style = "border:2px solid black" ></hr>

<div class="alert alert-info">
<font color=black>

- **LightFM** provides a function for fetching the **MovieLens 100K dataset**
- this small recommender dataset, consisting of around 950 users, 1700 movies, and 100,000 ratings. 
- The ratings are on a scale from 1 to 5, but we'll all treat them as implicit positive feedback in this example. 

</font>
</div>

In [3]:
movielens = fetch_movielens()
train = movielens['train']
test = movielens['test']
train

<943x1682 sparse matrix of type '<class 'numpy.int32'>'
	with 90570 stored elements in COOrdinate format>

# BPR vs. WARP
<hr style = "border:2px solid black" ></hr>

<div class="alert alert-info">
<font color=black>

- Compares the models' performance between a model that uses two losses: BPR and WARP.
- We'll use two ranking metrics to evaluate the performance: precision@k and ROC AUC. 
- For precision at k we'll be looking at whether they are within the first k = 10 results on the list
- For AUC, we'll be calculating the probability that any known positive is higher on the list than a random negative example.

</font>
</div>

In [4]:
start = time()
model = LightFM(learning_rate=0.05, loss='bpr')
model.fit(train, epochs=50)
print(time() - start)

train_precision = precision_at_k(model, train, k=10).mean()
test_precision = precision_at_k(model, test, k=10).mean()

train_auc = auc_score(model, train).mean()
test_auc = auc_score(model, test).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))

3.189851999282837
Precision: train 0.63, test 0.09.
AUC: train 0.92, test 0.87.


In [5]:
start = time()
model = LightFM(learning_rate=0.05, loss='warp')
model.fit(train, epochs=50)
print(time() - start)

train_precision = precision_at_k(model, train, k=10).mean()
test_precision = precision_at_k(model, test, k=10).mean()

train_auc = auc_score(model, train).mean()
test_auc = auc_score(model, test).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))

3.230599880218506
Precision: train 0.65, test 0.11.
AUC: train 0.95, test 0.91.


# Conclusions
<hr style = "border:2px solid black" ></hr>

<div class="alert alert-danger">
<font color=black>

- Despite the fact that WARP optimizes for precision@k its performance, it is able to obtain both a higher recision and ROC AUC score. 
- However, this is not a **general result**.
- **Nevertheless**, in this case we can infer that taking the extra step to ensure the sampled negative item is violating the pairwise comparison and penalizing the loss function more if we were able to find a violating negative item quickly does in fact gives performance boost. 

</font>
</div>

# References
<hr style = "border:2px solid black" ></hr>

<div class="alert alert-warning">
<font color=black>

- [Youtube: An Introduction to the Geometric Distribution](https://www.youtube.com/watch?v=zq9Oz82iHf0)
- [Blog: WARP loss for implicit-feedback recommendation](http://building-babylon.net/2016/03/18/warp-loss-for-implicit-feedback-recommendation/)
- [Jupyter Notebook: Learning-to-rank using the WARP loss](http://nbviewer.jupyter.org/github/lyst/lightfm/blob/master/examples/movielens/warp_loss.ipynb)
- [Jupyter Notebook: An implicit feedback recommender for the Movielens dataset](http://nbviewer.jupyter.org/github/lyst/lightfm/blob/master/examples/movielens/example.ipynb)
- [Paper: J. Weston, S. Bengio, N. Usunier - Wsabie: Scaling up to large vocabulary image annotation (2011)](http://www.thespermwhale.com/jaseweston/papers/wsabie-ijcai.pdf)
- [This notebook](https://nbviewer.jupyter.org/github/ethen8181/machine-learning/blob/master/recsys/5_warp.ipynb)
- [lightfm library](https://github.com/lyst/lightfm)
- [MovieLens 100K dataset](https://grouplens.org/datasets/movielens/100k/)

</font>
</div>